# Comparación búsquedas aproximadas con k-d tree y k-means tree

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025  

### Gráficos interactivos

Para los gráficos se usa matplotlib:
```
pip install matplotlib
```

Para gráficos interactivos (por ej. hacer zoom):

  1. Instalar ipympl:  `pip install ipympl`
  2. Reiniciar jupyter 
  3. Reemplazar `%matplotlib inline` por `%matplotlib widget`


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Descomentar esta linea para graficos interactivos
## %matplotlib widget


### Clases para hacer los experimentos y la comparación

In [ ]:
import numpy
import time
import pyflann


class Curva:
    def __init__(self):
        self.precisiones = []
        self.eficiencias = []


class Experimento:
    def __init__(self, nombre, archivo_dataset_q, archivo_dataset_r):
        self.nombre = nombre
        # cargar conjuntos de vectores Q y R
        self.dataset_q = numpy.load(archivo_dataset_q)
        self.dataset_r = numpy.load(archivo_dataset_r)
        print(
            "{}: conjunto_Q={} conjunto_R={}".format(
                nombre, self.dataset_q.shape, self.dataset_r.shape
            )
        )
        # para crear indices
        self.flann = pyflann.FLANN()

    def linear_scan(self):
        print(
            "linear scan {} de Q={} x R={}".format(
                self.nombre, self.dataset_q.shape, self.dataset_r.shape
            )
        )
        self.flann.build_index(self.dataset_r, algorithm="linear")
        t0 = time.time()
        self.gt_nns, self.gt_dists = self.flann.nn_index(
            self.dataset_q, num_neighbors=1, cores=4
        )
        self.gt_segundos = time.time() - t0
        print("linear scan = {:.1f} segundos".format(self.gt_segundos))

    def evaluar_busqueda(self, nns, dists, tiempo):
        correctas = 0
        incorrectas = 0
        for i in range(len(self.gt_nns)):
            if nns[i] == self.gt_nns[i] or dists[i] == self.gt_dists[i]:
                correctas += 1
            else:
                incorrectas += 1
        precision = correctas / (correctas + incorrectas)
        eficiencia = tiempo / self.gt_segundos
        return precision, eficiencia

    def evaluar_arbol(self, algorithm, trees=0, branching=0):
        t0 = time.time()
        self.flann.build_index(
            self.dataset_r, algorithm=algorithm, trees=trees, branching=branching
        )
        nombre_arbol = "{}-{}".format(algorithm, max(trees, branching))
        print(
            "construcción {} = {:.2f} segundos".format(nombre_arbol, time.time() - t0)
        )
        curva = Curva()
        for checks in (1, 10, 20, 40, 60, 80, 100, 200, 400, 500, 600, 700, 800, 900, 1000, 2000, 4000):
            t0 = time.time()
            nn, dist = self.flann.nn_index(
                self.dataset_q, num_neighbors=1, cores=4, checks=checks
            )
            precision, eficiencia = self.evaluar_busqueda(nn, dist, time.time() - t0)
            print("  {:<10} precision={:>6.1%}  tiempo={:>5.1%}  checks={:>4}".format(
                    nombre_arbol, precision, eficiencia, checks) )
            curva.precisiones.append(precision)
            curva.eficiencias.append(eficiencia)
        return curva

# Parte 1: Comparación en Dataset_A


In [ ]:
ev1 = Experimento("DATASET_A", "dataset_a_q.npy", "dataset_a_r.npy")
ev1.linear_scan()
curva_kd1 = ev1.evaluar_arbol(algorithm="kdtree", trees=1)
curva_kd10 = ev1.evaluar_arbol(algorithm="kdtree", trees=10)
curva_kd30 = ev1.evaluar_arbol(algorithm="kdtree", trees=30)
curva_km3 = ev1.evaluar_arbol(algorithm="kmeans", branching=3)
curva_km30 = ev1.evaluar_arbol(algorithm="kmeans", branching=30)
curva_km90 = ev1.evaluar_arbol(algorithm="kmeans", branching=90)

### Graficar resultados k-d trees

In [ ]:

plt.plot(curva_kd1.precisiones,  curva_kd1.eficiencias,  label='kdtree-1',  color='r', linestyle='--', marker='o', markerfacecolor='c', markersize=8)
plt.plot(curva_kd10.precisiones, curva_kd10.eficiencias, label='kdtree-10', color='g', linestyle='--', marker='o', markerfacecolor='m', markersize=8)
plt.plot(curva_kd30.precisiones, curva_kd30.eficiencias, label='kdtree-30', color='b', linestyle='--', marker='o', markerfacecolor='y', markersize=8)
plt.plot(curva_km3.precisiones,  curva_km3.eficiencias,  label='kmeans-3',  color='c', linestyle='-.', marker='^', markerfacecolor='r', markersize=8)
plt.plot(curva_km30.precisiones, curva_km30.eficiencias, label='kmeans-30', color='m', linestyle='-.', marker='^', markerfacecolor='g', markersize=8)
plt.plot(curva_km90.precisiones, curva_km90.eficiencias, label='kmeans-90', color='y', linestyle='-.', marker='^', markerfacecolor='b', markersize=8)
plt.title(ev1.nombre)
plt.xlabel('Precisión NN [comparado con LS]')
plt.ylabel('Tiempo [comparado con LS]')
plt.xlim(0,1)
plt.ylim(0,1)
plt.legend()
plt.plot([0.95, 0.95], [0.48, 0], linewidth=1, linestyle='-', marker=' ', color='r')
plt.text(0.88, 0.50, "95% precisión", ha='center', fontsize=9, color='red')
plt.plot([0.5, 0.5], [0.48, 0], linewidth=1, linestyle='-', marker=' ', color='r')
plt.text(0.5, 0.5, "50% precisión", ha='center', fontsize=9, color='red')
plt.show()

# Parte 2: Repetir el experimento con Dataset_B

In [ ]:
ev2 = Experimento("DATASET_B", "dataset_b_q.npy", "dataset_b_r.npy")
ev2.linear_scan()
curva2_kd1 = ev2.evaluar_arbol(algorithm="kdtree", trees=1)
curva2_kd10 = ev2.evaluar_arbol(algorithm="kdtree", trees=10)
curva2_kd30 = ev2.evaluar_arbol(algorithm="kdtree", trees=30)
curva2_km3 = ev2.evaluar_arbol(algorithm="kmeans", branching=3)
curva2_km30 = ev2.evaluar_arbol(algorithm="kmeans", branching=30)
curva2_km90 = ev2.evaluar_arbol(algorithm="kmeans", branching=90)

### Graficar resultados

In [ ]:
plt.plot(
    curva2_kd1.precisiones,
    curva2_kd1.eficiencias,
    label="kdtree-1",
    color="r",
    linestyle="--",
    marker="o",
    markerfacecolor="c",
    markersize=8,
)
plt.plot(
    curva2_kd10.precisiones,
    curva2_kd10.eficiencias,
    label="kdtree-10",
    color="g",
    linestyle="--",
    marker="o",
    markerfacecolor="m",
    markersize=8,
)
plt.plot(
    curva2_kd30.precisiones,
    curva2_kd30.eficiencias,
    label="kdtree-30",
    color="b",
    linestyle="--",
    marker="o",
    markerfacecolor="y",
    markersize=8,
)
plt.plot(
    curva2_km3.precisiones,
    curva2_km3.eficiencias,
    label="kmeans-3",
    color="c",
    linestyle="-.",
    marker="^",
    markerfacecolor="r",
    markersize=8,
)
plt.plot(
    curva2_km30.precisiones,
    curva2_km30.eficiencias,
    label="kmeans-30",
    color="m",
    linestyle="-.",
    marker="^",
    markerfacecolor="g",
    markersize=8,
)
plt.plot(
    curva2_km90.precisiones,
    curva2_km90.eficiencias,
    label="kmeans-90",
    color="y",
    linestyle="-.",
    marker="^",
    markerfacecolor="b",
    markersize=8,
)
plt.title(ev2.nombre)
plt.xlabel("Precisión NN [comparado con LS]")
plt.ylabel("Tiempo [comparado con LS]")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.plot([0.95, 0.95], [0.48, 0], linewidth=1, linestyle="-", marker=" ", color="r")
plt.text(0.88, 0.5, "95% precisión", ha="center", fontsize=9, color="red")
plt.plot([0.5, 0.5], [0.48, 0], linewidth=1, linestyle="-", marker=" ", color="r")
plt.text(0.5, 0.5, "50% precisión", ha="center", fontsize=9, color="red")
plt.show()

En los gráficos anteriores se muestra una comparación de k-d trees y k-means trees con varios parámetros. Al igual que en el anexo anterior, es interesante identificar la configuración (índice y número de checks) que logra un 95% de precisión en el menor tiempo.

**Ejercicio:** Con los datos obtenidos, identificar la configuración que logra en el menor tiempo una precisión de 50% y una precisión de 95%. **¿Es la misma configuración en un mismo dataset? ¿Es la misma configuración en ambos datasets?**